In [0]:
import sys
sys.path.insert(0, "../lib/")
import ingestores
import utils


tabela = dbutils.widgets.get("tabela")
#tabela ="cotacoes"
schema = "cotacoes_silver"
catalog = "prod"
checkpointpath = f"/Volumes/transacional/cotacoes_db/checkpoint/checkpoint_silver_{tabela}"
dbpath = f"prod.cotacoes_silver.{tabela}"

query = utils.import_query(f"{tablename}.sql")
if not spark.catalog.tableExists(f"{catalog}.{schema}.{tabela}"):

        print("Tabela não existente, criando.")
        dbutils.fs.rm(checkpointpath, recurse=True)
        (spark.sql(query)
              .writeTo(f"{self.catalog}.{self.schema}.{self.tabela}")   
              .tableProperty("delta.autoOptimize.optimizeWrite", "true")
              .tableProperty("delta.autoOptimize.autoCompact", "true")
              .tableProperty("delta.enableChangeDataFeed", "true")
              .using("delta") 
              .createOrReplace()
              )     
        
else:
        print("Tabela já existente, ignorando FULL LOAD")
        ingestao_cdc = ingestores.ingestaoCDC(spark, tabela, schema, catalog, dbpath, checkpointpath)
        ingestao_cdc.runCDC()